## Netcat - dd_udp.c - 16 MiB rcv sckt buffer

In [17]:
import pandas as pd

tests1 = pd.read_csv('https://raw.githubusercontent.com/Gert-Jan1966/DataDiodesAndPacketLoss/main/Testresults/recbuff-16mib/dd_udp.c/nc/testresults_nc_100000000.csv')
tests2 = pd.read_csv('https://raw.githubusercontent.com/Gert-Jan1966/DataDiodesAndPacketLoss/main/Testresults/recbuff-16mib/dd_udp.c/nc/testresults_nc_250000000.csv')
tests3 = pd.read_csv('https://raw.githubusercontent.com/Gert-Jan1966/DataDiodesAndPacketLoss/main/Testresults/recbuff-16mib/dd_udp.c/nc/testresults_nc_500000000.csv')
tests4 = pd.read_csv('https://raw.githubusercontent.com/Gert-Jan1966/DataDiodesAndPacketLoss/main/Testresults/recbuff-16mib/dd_udp.c/nc/testresults_nc_750000000.csv')
tests5 = pd.read_csv('https://raw.githubusercontent.com/Gert-Jan1966/DataDiodesAndPacketLoss/main/Testresults/recbuff-16mib/dd_udp.c/nc/testresults_nc_1000000000.csv')
results = pd.concat([tests1, tests2, tests3, tests4, tests5])
results.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   bitrate            5000 non-null   int64  
 1   trial              5000 non-null   int64  
 2   sender_rc          5000 non-null   int64  
 3   receiver_rc        5000 non-null   int64  
 4   Xfer_time          5000 non-null   float64
 5   Xfer_success       5000 non-null   bool   
 6   in_file_size       5000 non-null   int64  
 7   out_file_size      5000 non-null   int64  
 8   dtgrms_sent        5000 non-null   int64  
 9   dtgrms_received    5000 non-null   int64  
 10  dtgrms_corrupt     5000 non-null   object 
 11  tx_errors          5000 non-null   int64  
 12  tx_dropped         5000 non-null   int64  
 13  rx_errors          5000 non-null   int64  
 14  rx_dropped         5000 non-null   int64  
 15  snmp_OutDatagrams  5000 non-null   int64  
 16  snmp_SndbufErrors  5000 non-nu

#### Transfer success vs Transfer failure

In [18]:
results['Xfer_success'].value_counts()

Xfer_success
True     4798
False     202
Name: count, dtype: int64

#### Did we lose Frames at the Link layer?
In that case, at least one of 'tx_errors', 'tx_dropped', 'rx_errors' or 'rx_dropped' should not be zero.....

In [19]:
results.query('(tx_errors != 0) | (tx_dropped != 0) | (rx_errors != 0) | (rx_dropped != 0)').value_counts()

Series([], Name: count, dtype: int64)

#### Did we lose UDP Datagrams at the Transport layer? (needs dd_udp.c module)

In [20]:
results.query('dtgrms_sent != dtgrms_received').value_counts()

Series([], Name: count, dtype: int64)

#### How many trials with receiver socket buffer errors? (lost Segments)

In [21]:
results.query('snmp_RcvbufErrors != 0').value_counts()

bitrate     trial  sender_rc  receiver_rc  Xfer_time  Xfer_success  in_file_size  out_file_size  dtgrms_sent  dtgrms_received  dtgrms_corrupt  tx_errors  tx_dropped  rx_errors  rx_dropped  snmp_OutDatagrams  snmp_SndbufErrors  snmp_InDatagrams  snmp_InErrors  snmp_RcvbufErrors
1000000000  939    0          0            9.243989   False         125000000     124098880      7630         7630             na              0          0           0          0           7630               0                  7575              55             55                   1
100000000   4      0          0            11.037621  False         125000000     104274240      7630         7630             na              0          0           0          0           7630               0                  6365              1265           1265                 1
750000000   587    0          0            2.379142   False         125000000     89128960       7630         7630             na              0          0 

#### Does the number of receiver socket InErrors equal the number of receiver buffer errors?

In [22]:
results.query('snmp_InErrors != snmp_RcvbufErrors').value_counts()

Series([], Name: count, dtype: int64)

#### Do all unsuccessful trials match all trials with receiver buffer errors?

In [23]:
results.query('(snmp_RcvbufErrors != 0) & (Xfer_success == False)').value_counts()

bitrate     trial  sender_rc  receiver_rc  Xfer_time  Xfer_success  in_file_size  out_file_size  dtgrms_sent  dtgrms_received  dtgrms_corrupt  tx_errors  tx_dropped  rx_errors  rx_dropped  snmp_OutDatagrams  snmp_SndbufErrors  snmp_InDatagrams  snmp_InErrors  snmp_RcvbufErrors
1000000000  939    0          0            9.243989   False         125000000     124098880      7630         7630             na              0          0           0          0           7630               0                  7575              55             55                   1
100000000   4      0          0            11.037621  False         125000000     104274240      7630         7630             na              0          0           0          0           7630               0                  6365              1265           1265                 1
750000000   587    0          0            2.379142   False         125000000     89128960       7630         7630             na              0          0 

#### And what about snmp_SndbufErrors?

In [24]:
results.query('(snmp_SndbufErrors != 0)').value_counts()
#results.query('(snmp_SndbufErrors != 0) & (Xfer_success == True)').value_counts()

Series([], Name: count, dtype: int64)